In [2]:
import os
import urllib.request


def download_file(file_link, filename):
    # Checks if the file already exists before downloading
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(file_link, filename)
        print("File downloaded successfully.")
    else:
        print("File already exists.")

# Dowloading GGML model from HuggingFace
ggml_model_path = "https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q4_K_M.gguf"

filename = "input/mistral-7b-v0.1.Q4_K_M.gguf"

download_file(ggml_model_path, filename)

File already exists.


In [3]:
from llama_cpp import Llama

llm = Llama(model_path=filename, n_ctx=512, n_batch=126)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [102]:
def get_llm_generator(
    prompt,
    max_tokens=512,
    temperature=0,
    top_p=0.9,
    echo=False,
    stop=["[INST]"],
):
    text_generator = llm(
        prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        echo=echo,
        stream=True,
        stop=stop,
    )
    return text_generator

def get_llm_generation(
    prompt,
    max_tokens=512,
    temperature=0,
    top_p=0.9,
    echo=False,
    stop=["[INST]"],
):
    text_generation = llm(
        prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        echo=echo,
        stop=stop,
    )
    return text_generation["choices"][0]["text"].strip()

def generate_chat_prompt(input):
    system = "You are a helpful bot that answers any questions the user may have. Only answer in short clear sentences."
    chat_prompt_template = f"<s>[INST] {system} [/INST]</s>{input}"
    return chat_prompt_template

def generate_skill_extraction_prompt(input):
    system = "You are a helpful bot that extracts a couple of skills the user will learn after learning what the user wants to learn. Only answer in a list of comma separated words in between curly brackets."
    chat_prompt_template = f"<s>[INST] {system} [/INST]</s>{input}"
    return chat_prompt_template

def process_skill_extraction_prompt(input, prompt):
    if "{" in input and "}" in input:
        return input[input.find("{")+1:input.find("}")]
    else:
        return prompt

In [95]:
# Test User Help Generator
prompt = generate_chat_prompt(
    "How can I learn more about AI?"
)

generator = get_llm_generator(prompt)

buffer = []
buffer_size = 3

for word in generator:
    buffer.append(word["choices"][0]["text"].strip())
    if len(buffer) >= buffer_size:
        print(" ".join(buffer))
        buffer.clear()

Llama.generate: prefix-match hit


  You
can learn more
about AI by
reading books ,
taking courses ,
and attending con
ferences . There
are many resources
available online and
off line to
help you learn
more about AI
.  


In [105]:
# Test Skill Extraction
prompt = "I want to learn the most important languages in the world."
processed_prompt = generate_skill_extraction_prompt(
    prompt
)
output = get_llm_generation(processed_prompt)
processed_output = process_skill_extraction_prompt(output, prompt)
processed_output

Llama.generate: prefix-match hit


'English, Spanish, French, German, Italian, Portuguese, Russian, Chinese, Japanese, Korean'